---
## Gene Signature Scoring with Random Background signatures

L.Richards  
April 2020  

---

In [ ]:
#!/bin/bash
#SBATCH -t 72:00:00  #	<-- walltime for your job in format of days-hours:minutes:seconds
#SBATCH --mem=60G    #	<-- amount of memory your job needs to run, in megabytes by default
#SBATCH -p himem     #  <-- partition you job wants to run
#SBATCH -c 1 	     #  <-- number of CPUs your job needs to run
#SBATCH -N 1 	     #  <-- number of nodes your job needs to run. Please use 1 node unless you are running mpi jobs.

###########
### GSVA  on Random Gene Sigs
### L.Richards
### April 8, 2020
############

module load R/3.5.0

Rscript GSVA_RandomSigs.R $1


In [ ]:
library("Seurat", 
        lib="/cluster/projects/pughlab/projects/BTSCs_scRNAseq/Manuscript_G607removed/bin/"
       )

suppressPackageStartupMessages({
    #library(GSEABase)
    library(GSVA)
    #library(Biobase)
    library(RColorBrewer)
    library(parallel)
    #library(GSA)
    library(qvalue)
    library(taRifx)
    library(optparse)
})

###############
load.file <- args[1] ##path to random gene sig
print(load.file)

######
setwd("/cluster/home/lrichard/Branavan_NatComm/GSVA/Random_Sigs/")

print("Load Seurat Data")

### load in data and gene signatures
Wnt_medullo <- readRDS("/cluster/home/lrichard/Branavan_NatComm/GSVA/Hovestadt_Seurat_GSVA.rds")
dat <- as.matrix(Wnt_medullo@scale.data)

print("---------")
print(load.file)
print(Sys.time())
    
random.sigs <- readRDS(load.file)

print("Scoring random lists with GSVA")
print("Scoring on scaled data")
print(Sys.time())

##### GSVA on random sigs 
EnrichmentScores <- gsva(expr = dat, 
                         gset.idx.list = random.sigs, 
                         method = "gsva",
                         min.sz = 2, 
                         max.sz = Inf, 
                         mx.diff = TRUE, 
                         verbose = T, 
                         parallel.sz = 10
                        )
    
print("Saving GSVA results")
print(Sys.time())
    
filename <- gsub("_random_lists.rds", "_GSVA_SCORES.rds", load.file)
print(filename)
saveRDS(EnrichmentScores, file = filename)
    
print("")
print("")
print("END")
print(Sys.time())